In [1]:
import re
import pandas as pd
import numpy as np
import joblib

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import gensim
import nltk

from nltk.stem.snowball import SnowballStemmer

from gensim.corpora import Dictionary, MmCorpus
from gensim.models import ldamodel

from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.decomposition import PCA

import os
from gensim.test.utils import datapath

import gensim
from gensim import corpora
from gensim import models
from collections import Counter
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
from collections import Counter

PATH = "D:/Petakabar/what/"
prebandingan_path = "D:/Petakabar/perbandingan/"
bencana = "bencana/"
ekonomi = "ekonomi/"
kecelakaan = "kecelakaan/"
kesehatan = "kesehatan/"
kriminalitas = "kriminalitas/"
olahraga = "olahraga/"

news = pd.read_csv('perbandingan/berita-24-5.csv')

is_train = False
is_perbandingan = True

In [2]:
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')

from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

def remove_number(text):
    return  re.sub(r"\d+", "", text)

def preprocessing(berita, mode='default', topik='default'):
    NLTK_StopWords = stopwords.words('indonesian')
    # print(NLTK_StopWords)
    NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali",
                        "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "allah", "bismillah"])
    NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                        'kalo', 'amp', 'biar', 'bikin', 'bilang',
                        'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                        'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                        'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                        'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                        '&amp', 'yah'])

    if mode == 'lda':
        if topik == bencana[:-1]:
            txt_stopword = pd.read_csv(PATH + bencana + "stopwords_add_v1.txt", names= ["stopwords"], header = None)
            NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
            txt_stopword = pd.read_csv(PATH + bencana + "stopwords_add_v2.txt", names= ["stopwords"], header = None)
            NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
            txt_stopword = pd.read_csv(PATH + bencana + "stopwords_add_v3.txt", names= ["stopwords"], header = None)
            NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
        if topik == ekonomi[:-1]:
            txt_stopword = pd.read_csv(PATH + ekonomi + "stopwords_add_v1.txt", names= ["stopwords"], header = None)
            NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
        if topik == kecelakaan[:-1]:
            txt_stopword = pd.read_csv(PATH + kecelakaan + "stopwords_add_v1.txt", names= ["stopwords"], header = None)
            NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
        if topik == kesehatan[:-1]:
            txt_stopword = pd.read_csv(PATH + kesehatan + "stopwords_add_v1.txt", names= ["stopwords"], header = None)
            NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
        if topik == kriminalitas[:-1]:
            txt_stopword = pd.read_csv(PATH + kriminalitas + "stopwords_add_v1.txt", names= ["stopwords"], header = None)
            NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
        if topik == olahraga[:-1]:
            txt_stopword = pd.read_csv(PATH + olahraga + "stopwords_add_v1.txt", names= ["stopwords"], header = None)
            NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))

    NLTK_StopWords = set(NLTK_StopWords)
    s = str(berita)
    s = s.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    if mode == 'lda':
        # for lda ----
        s = re.sub(r'[^a-zA-Z\s]', ' ', s)
        # ------------
    else:
        s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
                    
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if (t not in NLTK_StopWords)]
    return T

In [4]:
from gensim.models import Phrases
from datetime import datetime

iter_through = [
    'bencana',
    'ekonomi',
    'kecelakaan',
    'kesehatan',
    'kriminalitas',
    'olahraga'
    ]

total_topics = [
    4,
    5,
    5,
    5,
    5,
    4
]

number_words = [
    29,
    29,
    29,
    29,
    29,
    26
]

for topik_id, n in enumerate(iter_through):
    if is_train == False:
        break
    df_train = joblib.load('D:/Petakabar/perbandingan/' + n + '_train_unpreprocessed.pkl')

    df_train = df_train[['berita_title', 'berita_date', 'berita_desc', 'berita_source']]
    df_train.rename(columns = {'berita_title':'title', 'berita_date':'date', 'berita_desc':'desc', 'berita_source':'source'}, inplace=True)
    
    print(len(df_train))

    for x in range(0, df_train.shape[0]):
        text = preprocessing(df_train.iloc[x, -2], mode = 'lda', topik = n)
        df_train.iloc[x, -2] = (' '.join(text))

    for x in range(0, df_train.shape[0]):
        text = preprocessing(df_train.iloc[x, -4], mode = 'lda', topik = n)
        df_train.iloc[x, -4] = (' '.join(text))

    df_train['desc'] = df_train['desc'].apply(remove_number)
    df_train['desc'] = df_train['desc'].apply(word_tokenize)
    df_train['desc'] = [[token for token in doc if len(token) > 2] for doc in df_train['desc']]
    df_train['title'] = df_train['title'].apply(remove_number)
    df_train['title'] = df_train['title'].apply(word_tokenize)
    df_train['title'] = [[token for token in doc if len(token) > 2] for doc in df_train['title']]
    # df_train['fdist'] = df_train['desc'].apply(FreqDist)

    bigram = Phrases(df_train['desc'], min_count=10)
    for idx in range(len(df_train['desc'])):
        for token in bigram[df_train['desc'].iloc[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                df_train['desc'].iloc[idx].append(token)
                word1, word2 = token.split("_")
                dont_remove = ('kereta', 'bencana', 'gempa', 'tsunami', 'meletus', 'getaran', 'vulkanik', 'meledak', 'ledakan', 'kebakaran',
                    'terbakar', 'banjir', 'abrasi', 'amblas', 'kekeringan', 'puting', 'beliung', 'angin', 'anjlok', 'pergerakan', 'tumbang',
                    'kencang', 'petir', 'lebat', 'hujan', 'longsor', 'genangan', 'meteor', 'meluap', 'dagang', 'berita', 'perdagangan', 'ekonomi',
                    'miliar', 'neraca', 'impor', 'resesi', 'persen', 'perekonomian', 'keuangan', 'sosio', 'makro', 'bisnis', 'moneter', 'makroekonomi',
                    'perekonomiannya', 'kesejahteraan', 'pertumbuhan', 'inflasi', 'industrialisasi', 'produktivitas', 'infrastruktur', 'pasar',
                    'produk', 'pembangunan', 'usaha', 'saham', 'ukm', 'ekspor', 'dieskpor', 'diimpor', 'mengekspor', 'mengimpor', 'pertumbuhan', 
                    'kecelakaan', 'tertabrak', 'tabrakan', 'menabrak', 'ditabrak', 'tenggelam', 'pesawat', 'simtoma', 'penyakit', 'kesehatan', 'covid',
                    'korban', 'pasien', 'terjangkit', 'tuberkulosis', 'dbd', 'hiv', 'diabetes', 'hipertensi', 'ginjal', 'jantung', 'melitus', 'infeksi',
                    'terinfeksi', 'bakteri', 'virus', 'vaksin', 'polio', 'imunisasi', 'meningitis', 'aids', 'diare', 'pneumonia', 'radang', 'pandemi',
                    'corona', 'malaria', 'muntah', 'kejang', 'batuk', 'hipertensi', 'obesitas', 'gizi', 'sars', 'darah', 'pasien', 'influenza', 'flu',
                    'kanker', 'anemia', 'hepatitis', 'liver', 'iritasi', 'pengobatan', 'obat', 'alergi', 'gatal', 'ruam', 'kelainan', 'bunuh', 'pembunuhan',
                    'membunuh', 'dibunuh', 'pencurian', 'curi', 'mencuri', 'dicuri', 'pemerkosaan', 'memperkosa', 'diperkosa', 'perkosa', 'rampok',
                    'perampokan', 'merampok', 'dirampok', 'begal', 'pembegalan', 'membegal', 'dibegal', 'pembacokan', 'bacok', 'membacok', 'dibacok',
                    'penusukan', 'tusuk', 'ditusuk', 'menusuk', 'pukul', 'pemukulan', 'memukul', 'dipukul', 'luka', 'melukai', 'dilukai', 'sepakbola',
                    'bulutangkis', 'basket', 'raket', 'voli', 'futsal', 'tenis', 'turnamen', 'motogp', 'pertandingan', 'pon')
                if word1 not in dont_remove:
                    df_train['desc'].iloc[idx].remove(word1)
                if word2 not in dont_remove:
                    df_train['desc'].iloc[idx].remove(word2)
                # print(token)
    print(dont_remove)
    dictionary = corpora.Dictionary(df_train['desc'])
    dictionary.filter_extremes(no_below=30, no_above=0.85)
    print(dictionary)

    corpus = [dictionary.doc2bow(berita) for berita in df_train['desc']]

    # Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel

    print(topik_id, total_topics[topik_id], number_words[topik_id])
    
    the_path = PATH + n + "/model/lda_model" + "_" + str(total_topics[topik_id]) + "_" + str(number_words[topik_id])
    if not os.path.exists(the_path):
        os.mkdir(the_path)
    temp_file = datapath(the_path + "/lda_model")
    lda_model = Lda(corpus, num_topics=total_topics[topik_id], id2word=dictionary, passes=20, iterations=100,alpha=[0.01]*total_topics[topik_id], eta=[0.01]*len(dictionary.keys()), random_state=1)
    lda_model.save(temp_file)

    lda_model.show_topics(num_topics=total_topics[topik_id], num_words=number_words[topik_id])

    now = datetime.now() # current date and time
    date_time = now.strftime("%m-%d-%Y_%H-%M-%S")
    print("date and time:",date_time)
    LDAvis_html = '/ldavis_prepared_'+ str(total_topics[topik_id])+"_"+str(number_words[topik_id])+"_"+str(date_time)
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    with open(PATH + n + LDAvis_html, 'wb') as f:
            pickle.dump(LDAvis_prepared, f)
    pyLDAvis.save_html(LDAvis_prepared, PATH + n + LDAvis_html +'.html')

    #Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=df_train['desc'], dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence Score:', coherence_lda)
    print('Perplexity Score:', lda_model.log_perplexity(corpus))
    

    # print(df_train.head(2))

In [4]:
from gensim.models import Phrases

iter_through = [
    'bencana',
    'ekonomi',
    'kecelakaan',
    'kesehatan',
    'kriminalitas',
    'olahraga'
    ]

total_topics = [
    5,
    7,
    6,
    6,
    7,
    5
]

number_words = [
    24,
    18,
    20,
    20,
    18,
    24
]

for topik_id, n in enumerate(iter_through):
    df_train = joblib.load('D:/Petakabar/perbandingan/' + n + '_test_unpreprocessed.pkl')

    df_train = df_train[['berita_title', 'berita_date', 'berita_desc', 'berita_source']]
    df_train.rename(columns = {'berita_title':'title', 'berita_date':'date', 'berita_desc':'desc', 'berita_source':'source'}, inplace=True)
    
    # df_train = df_train.iloc[0:1]
    # print(df_train)
    
    print(len(df_train))

    for x in range(0, df_train.shape[0]):
        text = preprocessing(df_train.iloc[x, -2], mode = 'lda', topik = n)
        df_train.iloc[x, -2] = (' '.join(text))

    for x in range(0, df_train.shape[0]):
        text = preprocessing(df_train.iloc[x, -4], mode = 'lda', topik = n)
        df_train.iloc[x, -4] = (' '.join(text))

    df_train['desc'] = df_train['desc'].apply(remove_number)
    df_train['desc'] = df_train['desc'].apply(word_tokenize)
    df_train['desc'] = [[token for token in doc if len(token) > 2] for doc in df_train['desc']]
    df_train['title'] = df_train['title'].apply(remove_number)
    df_train['title'] = df_train['title'].apply(word_tokenize)
    df_train['title'] = [[token for token in doc if len(token) > 2] for doc in df_train['title']]
    # df_train['fdist'] = df_train['desc'].apply(FreqDist)

    bigram = Phrases(df_train['desc'], min_count=4)
    for idx in range(len(df_train['desc'])):
        for token in bigram[df_train['desc'].iloc[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                df_train['desc'].iloc[idx].append(token)
                word1, word2 = token.split("_")
                dont_remove = ('kereta', 'bencana', 'gempa', 'tsunami', 'meletus', 'getaran', 'vulkanik', 'meledak', 'ledakan', 'kebakaran',
                    'terbakar', 'banjir', 'abrasi', 'amblas', 'kekeringan', 'puting', 'beliung', 'angin', 'anjlok', 'pergerakan', 'tumbang',
                    'kencang', 'petir', 'lebat', 'hujan', 'longsor', 'genangan', 'meteor', 'meluap', 'dagang', 'berita', 'perdagangan', 'ekonomi',
                    'miliar', 'neraca', 'impor', 'resesi', 'persen', 'perekonomian', 'keuangan', 'sosio', 'makro', 'bisnis', 'moneter', 'makroekonomi',
                    'perekonomiannya', 'kesejahteraan', 'pertumbuhan', 'inflasi', 'industrialisasi', 'produktivitas', 'infrastruktur', 'pasar',
                    'produk', 'pembangunan', 'usaha', 'saham', 'ukm', 'ekspor', 'dieskpor', 'diimpor', 'mengekspor', 'mengimpor', 'pertumbuhan', 
                    'kecelakaan', 'tertabrak', 'tabrakan', 'menabrak', 'ditabrak', 'tenggelam', 'pesawat', 'simtoma', 'penyakit', 'kesehatan', 'covid',
                    'korban', 'pasien', 'terjangkit', 'tuberkulosis', 'dbd', 'hiv', 'diabetes', 'hipertensi', 'ginjal', 'jantung', 'melitus', 'infeksi',
                    'terinfeksi', 'bakteri', 'virus', 'vaksin', 'polio', 'imunisasi', 'meningitis', 'aids', 'diare', 'pneumonia', 'radang', 'pandemi',
                    'corona', 'malaria', 'muntah', 'kejang', 'batuk', 'hipertensi', 'obesitas', 'gizi', 'sars', 'darah', 'pasien', 'influenza', 'flu',
                    'kanker', 'anemia', 'hepatitis', 'liver', 'iritasi', 'pengobatan', 'obat', 'alergi', 'gatal', 'ruam', 'kelainan', 'bunuh', 'pembunuhan',
                    'membunuh', 'dibunuh', 'pencurian', 'curi', 'mencuri', 'dicuri', 'pemerkosaan', 'memperkosa', 'diperkosa', 'perkosa', 'rampok',
                    'perampokan', 'merampok', 'dirampok', 'begal', 'pembegalan', 'membegal', 'dibegal', 'pembacokan', 'bacok', 'membacok', 'dibacok',
                    'penusukan', 'tusuk', 'ditusuk', 'menusuk', 'pukul', 'pemukulan', 'memukul', 'dipukul', 'luka', 'melukai', 'dilukai', 'sepakbola',
                    'bulutangkis', 'basket', 'raket', 'voli', 'futsal', 'tenis', 'turnamen', 'motogp', 'pertandingan', 'pon')
                if word1 not in dont_remove:
                    df_train['desc'].iloc[idx].remove(word1)
                if word2 not in dont_remove:
                    df_train['desc'].iloc[idx].remove(word2)
                # print(token)

    dictionary = corpora.Dictionary(df_train['desc'])
    dictionary.filter_extremes(no_below=30, no_above=0.85)
    print(dictionary)

    corpus = [dictionary.doc2bow(berita) for berita in df_train['desc']]

    # Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel

    print(topik_id, total_topics[topik_id], number_words[topik_id])
    
    # the_path = PATH + n + "/model/lda_model" + "_" + str(total_topics[topik_id]) + "_" + str(number_words[topik_id]) #old
    the_path = PATH + n + "/model/vis_lda_model" + "_" + str(total_topics[topik_id]) + "_0.01_0.01" #new
    if not os.path.exists(the_path):
        os.mkdir(the_path)
    temp_file = datapath(the_path + "/lda_model")
    lda_model = models.ldamodel.LdaModel.load(temp_file)

    topics = lda_model.show_topics(formatted=False, num_topics=total_topics[topik_id], num_words=number_words[topik_id])
    # lda_model.show_topics(num_topics=total_topics[topik_id], num_words=number_words[topik_id])
    # print('Perplexity Score:', lda_model.log_perplexity(corpus)) 
    # topic = lda_model.show_topics(num_topics=total_topics[topik_id], num_words=number_words[topik_id])
    # print((topics[0]))
    # print((topic[0]))
    data_flat = [w for w_list in df_train['desc'] for w in w_list]
    counter = Counter(data_flat)

    out = []
    for i, topic in topics:
        for word, weight in topic:
            out.append([word, i , weight, counter[word]])

    df_imp_wcount = pd.DataFrame(out, columns=['word', 'id', 'importance', 'word_count']) 
    # print(df_imp_wcount)

    def format_topics_sentences(ldamodel=None, corpus=corpus, texts=df_train['desc']):
        # Init output
        sent_topics_df = pd.DataFrame()
        # print(len(ldamodel[corpus]))

        dom = []
        perc = []
        keywords = []
        # Get main topic in each document
        for i, row_list in enumerate(ldamodel[corpus]):
            row = row_list[0] if ldamodel.per_word_topics else row_list            
            row = sorted(row, key=lambda x: (x[1]), reverse=True)
            # print(len(row))

            # Get the Dominant topic, Perc Contribution and Keywords for each document
            for j, (topic_num, prop_topic) in enumerate(row):
                if j == 0:  # => dominant topic
                    wp = ldamodel.show_topic(topic_num, topn=number_words[topik_id])
                    topic_keywords = ", ".join([word for word, prop in wp])
                    dom.append(int(topic_num))
                    perc.append(round(prop_topic,4))
                    keywords.append(topic_keywords)
                    # sent_topics_df = pd.concat([sent_topics_df, pd.DataFrame([pd.Series([int(topic_num), round(prop_topic,4), topic_keywords])])], ignore_index=True)
                    # sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                else:
                    break
        # print(len(sent_topics_df))
        sent_topics_df['Dominant_Topic'] = dom
        sent_topics_df['Perc_Contribution'] = perc
        sent_topics_df['Topic_Keywords'] = keywords
        # sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

        # Add original text to the end of the output
        # sent_topics_df['Texts']= texts
        # print(len(sent_topics_df))
        return(sent_topics_df)
    
    df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=df_train['desc'])
    df_topic_sents_keywords['Texts'] = 0
    df_topic_sents_keywords['Title'] = 0

    df_dominant_topic = df_topic_sents_keywords.reset_index()
    # texts = []
    for i in range(len(df_train['desc'])):
        # print(df_train['desc'].iloc[i])
        # texts.append(df_train['desc'].iloc[i])
        df_dominant_topic['Texts'].iloc[i] = df_train['desc'].iloc[i]
        df_dominant_topic['Title'].iloc[i] = df_train['title'].iloc[i]
    # df_dominant_topic['Texts'] = texts
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'Title']
    df_dominant_topic.to_csv(PATH + n + '/df_dominant_topic.csv')

    words = [word for i, topic in topics for word, weight in topic]
    word_topics = set(words)
    lda_words = [x.replace('_', ' ') for x in word_topics]
    # lda_words = list(word_topics)
    with open(PATH + n + "/lda_words.txt", 'w') as f:
            f.write(", ".join(lda_words))


    lda_what = []
    for i in range(len(df_train)):
        val = []
        for idx, key in enumerate(word_topics):
            if key in df_train['desc'].iloc[i]:
                val.append(key)
        lda_what.append(val)

    # print(lda_what)

    lda_title_what = []
    for i in range(len(df_train)):
        val = []
        for idx, key in enumerate(word_topics):
            if key in df_train['title'].iloc[i]:
                val.append(key)
        lda_title_what.append(val)

    # print(lda_title_what)

    for idx, word_list in enumerate(lda_title_what):
        val = []
        for word in word_list:
            if word in lda_what[idx]:
                continue
            val.append(word)
        lda_what[idx].extend(val)
        lda_what[idx] = set(lda_what[idx])
        lda_what[idx] = list(lda_what[idx])
        # if val:
        #     print(idx)
        #     print(val)
    # print(lda_what)


    test_news = joblib.load('D:/Petakabar/perbandingan/' + n + '_test_unpreprocessed.pkl')
    before_what = test_news[test_news['berita_topik_id'] == topik_id+1]
    before_what = before_what[['berita_title', 'qe_what', 'berita_desc']]
    df_result_lda = pd.DataFrame([0]*len(before_what), columns=['title_old'])
    df_result_lda['what_qe'] = 0
    df_result_lda['tp_qe'] = 0
    df_result_lda['tn_qe'] = 0
    df_result_lda['what_lda'] = 0
    df_result_lda['tp_lda'] = 0
    df_result_lda['tn_lda'] = 0
    df_result_lda['desc_old'] = 0
    print(len(before_what), len(df_result_lda))
    for i in range(len(df_result_lda)):
        print("-"*50)
        print(df_train['title'].iloc[i])
        print(df_train['desc'].iloc[i])
        print(lda_what[i])
        df_result_lda['what_lda'].iloc[i] = lda_what[i]
        df_result_lda['what_qe'].iloc[i] = before_what['qe_what'].iloc[i]
        df_result_lda['title_old'].iloc[i] = before_what['berita_title'].iloc[i]
        df_result_lda['desc_old'].iloc[i] = before_what['berita_desc'].iloc[i]

    # df_result_lda.to_csv(PATH + n + '/df_result_lda.csv')   
    # df_result_lda.to_excel(PATH + n + '/df_result_lda.xlsx', index=None) 
    df_result_lda

    # print(df_train.head(2))

208
Dictionary<24 unique tokens: ['angin', 'hujan', 'jawa', 'kencang', 'banjir']...>
0 5 24
208 208
--------------------------------------------------
['prakiraan', 'cuaca', 'jateng', 'solo', 'raya', 'hujan', 'lebat']
['jawa', 'diprakirakan', 'berpotensi', 'hujan', 'lebat', 'waspada', 'hidrometeorologi', 'jawa', 'dikeluarkan', 'meteorologi', 'berlaku', 'cerah', 'berawan', 'berawan', 'hujan', 'tersebar', 'merata', 'hujan', 'lebat', 'petir', 'angin', 'kencang', 'pegunungan', 'solo', 'raya', 'jateng', 'prakirawan', 'noor', 'jannah', 'cuacanya', 'cuaca', 'jateng', 'berlaku', 'hujan', 'sukoharjo', 'wonogiri', 'rembang', 'brebes', 'pekalongan', 'tegal', 'hujan', 'cilacap', 'kebumen', 'klaten', 'grobogan', 'blora', 'pati', 'demak', 'ungaran', 'pemalang', 'surakarta', 'salatiga', 'semarang', 'bumiayu', 'majenang', 'ambarawa', 'hujan', 'lebat', 'purwokerto', 'purbalingga', 'banjarnegara', 'kebumen', 'purworejo', 'wonosobo', 'mungkid', 'boyolali', 'karanganyar', 'sragen', 'kudus', 'jepara', 'tem

In [5]:
from gensim.models import Phrases
from datetime import datetime

iter_through = [
    'bencana',
    'ekonomi',
    'kecelakaan',
    'kesehatan',
    'kriminalitas',
    'olahraga'
    ]

total_topics = [
    4,
    5,
    5,
    5,
    5,
    4
]

number_words = [
    29,
    29,
    29,
    29,
    29,
    26
]

for topik_id, n in enumerate(iter_through):
    if is_perbandingan == False:
        break
    df_train = joblib.load('D:/Petakabar/perbandingan/' + n + '_train_unpreprocessed.pkl')
    # df_train = joblib.load('D:/Petakabar/perbandingan/' + n + '_test_unpreprocessed.pkl')

    df_train = df_train[['berita_title', 'berita_date', 'berita_desc', 'berita_source']]
    df_train.rename(columns = {'berita_title':'title', 'berita_date':'date', 'berita_desc':'desc', 'berita_source':'source'}, inplace=True)
    
    print(len(df_train))

    for x in range(0, df_train.shape[0]):
        text = preprocessing(df_train.iloc[x, -2], mode = 'lda', topik = n)
        df_train.iloc[x, -2] = (' '.join(text))

    for x in range(0, df_train.shape[0]):
        text = preprocessing(df_train.iloc[x, -4], mode = 'lda', topik = n)
        df_train.iloc[x, -4] = (' '.join(text))

    df_train['desc'] = df_train['desc'].apply(remove_number)
    df_train['desc'] = df_train['desc'].apply(word_tokenize)
    df_train['desc'] = [[token for token in doc if len(token) > 2] for doc in df_train['desc']]
    df_train['title'] = df_train['title'].apply(remove_number)
    df_train['title'] = df_train['title'].apply(word_tokenize)
    df_train['title'] = [[token for token in doc if len(token) > 2] for doc in df_train['title']]
    # df_train['fdist'] = df_train['desc'].apply(FreqDist)

    bigram = Phrases(df_train['desc'], min_count=10)
    for idx in range(len(df_train['desc'])):
        for token in bigram[df_train['desc'].iloc[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                df_train['desc'].iloc[idx].append(token)
                word1, word2 = token.split("_")
                dont_remove = ('kereta', 'bencana', 'gempa', 'tsunami', 'meletus', 'getaran', 'vulkanik', 'meledak', 'ledakan', 'kebakaran',
                    'terbakar', 'banjir', 'abrasi', 'amblas', 'kekeringan', 'puting', 'beliung', 'angin', 'anjlok', 'pergerakan', 'tumbang',
                    'kencang', 'petir', 'lebat', 'hujan', 'longsor', 'genangan', 'meteor', 'meluap', 'dagang', 'berita', 'perdagangan', 'ekonomi',
                    'miliar', 'neraca', 'impor', 'resesi', 'persen', 'perekonomian', 'keuangan', 'sosio', 'makro', 'bisnis', 'moneter', 'makroekonomi',
                    'perekonomiannya', 'kesejahteraan', 'pertumbuhan', 'inflasi', 'industrialisasi', 'produktivitas', 'infrastruktur', 'pasar',
                    'produk', 'pembangunan', 'usaha', 'saham', 'ukm', 'ekspor', 'dieskpor', 'diimpor', 'mengekspor', 'mengimpor', 'pertumbuhan', 
                    'kecelakaan', 'tertabrak', 'tabrakan', 'menabrak', 'ditabrak', 'tenggelam', 'pesawat', 'simtoma', 'penyakit', 'kesehatan', 'covid',
                    'korban', 'pasien', 'terjangkit', 'tuberkulosis', 'dbd', 'hiv', 'diabetes', 'hipertensi', 'ginjal', 'jantung', 'melitus', 'infeksi',
                    'terinfeksi', 'bakteri', 'virus', 'vaksin', 'polio', 'imunisasi', 'meningitis', 'aids', 'diare', 'pneumonia', 'radang', 'pandemi',
                    'corona', 'malaria', 'muntah', 'kejang', 'batuk', 'hipertensi', 'obesitas', 'gizi', 'sars', 'darah', 'pasien', 'influenza', 'flu',
                    'kanker', 'anemia', 'hepatitis', 'liver', 'iritasi', 'pengobatan', 'obat', 'alergi', 'gatal', 'ruam', 'kelainan', 'bunuh', 'pembunuhan',
                    'membunuh', 'dibunuh', 'pencurian', 'curi', 'mencuri', 'dicuri', 'pemerkosaan', 'memperkosa', 'diperkosa', 'perkosa', 'rampok',
                    'perampokan', 'merampok', 'dirampok', 'begal', 'pembegalan', 'membegal', 'dibegal', 'pembacokan', 'bacok', 'membacok', 'dibacok',
                    'penusukan', 'tusuk', 'ditusuk', 'menusuk', 'pukul', 'pemukulan', 'memukul', 'dipukul', 'luka', 'melukai', 'dilukai', 'sepakbola',
                    'bulutangkis', 'basket', 'raket', 'voli', 'futsal', 'tenis', 'turnamen', 'motogp', 'pertandingan', 'pon')
                if word1 not in dont_remove:
                    df_train['desc'].iloc[idx].remove(word1)
                if word2 not in dont_remove:
                    df_train['desc'].iloc[idx].remove(word2)
                # print(token)
    print(dont_remove)
    dictionary = corpora.Dictionary(df_train['desc'])
    dictionary.filter_extremes(no_below=30, no_above=0.85)
    print(dictionary)

    corpus = [dictionary.doc2bow(berita) for berita in df_train['desc']]

    # supporting function
    def compute_coherence_values(dictionary, k, a, b):
        
        lda_model = gensim.models.ldamodel.LdaModel( corpus,
                                            id2word=dictionary,
                                            num_topics=k, 
                                            random_state=1,
                                            passes=20,
                                            iterations=100,
                                            alpha=a,
                                            eta=b)
        
        the_path = PATH + n + "/model/vis_lda_model" + "_" + str(k) + "_" + str(a) + "_" + str(b)
        if not os.path.exists(the_path):
            os.mkdir(the_path)
        temp_file = datapath(the_path + "/lda_model")
        lda_model.save(temp_file)
        
        coherence_model_lda = CoherenceModel(model=lda_model, texts=df_train['desc'], dictionary=dictionary, coherence='c_v')
        
        return coherence_model_lda.get_coherence(), lda_model.log_perplexity(corpus)

    import tqdm

    # Topics range
    min_topics = 2
    max_topics = 8
    step_size = 1
    topics_range = range(min_topics, max_topics, step_size)

    # Alpha parameter
    alpha = [0.01, 0.1, 1]
    # alpha = list(np.arange(0.01, 1.02, 0.5))
    alpha.append('symmetric')
    # alpha.append('asymmetric')

    # Beta parameter
    beta = [0.01, 0.1, 1]
    # beta = list(np.arange(0.01, 1.02, 0.5))
    beta.append('symmetric')

    model_results = {
                    'Topics': [],
                    'Alpha': [],
                    'Beta': [],
                    'Coherence': [],
                    'Perplexity':[]
                    }

    # Can take a long time to run
    if 1 == 1:
        pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)))
        
        # iterate through validation corpuses
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta: 
                    # get the coherence score for the given parameters
                    res = compute_coherence_values(dictionary=dictionary, 
                                                k=k, a=a, b=b)
                    # Save the model results
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(res[0])
                    model_results['Perplexity'].append(res[1])
                    
                    pbar.update(1)
        pd.DataFrame(model_results).to_csv(prebandingan_path + 'lda_tuning_train_results_'+ n +'.csv', index=False)
        pbar.close()

2565
('kereta', 'bencana', 'gempa', 'tsunami', 'meletus', 'getaran', 'vulkanik', 'meledak', 'ledakan', 'kebakaran', 'terbakar', 'banjir', 'abrasi', 'amblas', 'kekeringan', 'puting', 'beliung', 'angin', 'anjlok', 'pergerakan', 'tumbang', 'kencang', 'petir', 'lebat', 'hujan', 'longsor', 'genangan', 'meteor', 'meluap', 'dagang', 'berita', 'perdagangan', 'ekonomi', 'miliar', 'neraca', 'impor', 'resesi', 'persen', 'perekonomian', 'keuangan', 'sosio', 'makro', 'bisnis', 'moneter', 'makroekonomi', 'perekonomiannya', 'kesejahteraan', 'pertumbuhan', 'inflasi', 'industrialisasi', 'produktivitas', 'infrastruktur', 'pasar', 'produk', 'pembangunan', 'usaha', 'saham', 'ukm', 'ekspor', 'dieskpor', 'diimpor', 'mengekspor', 'mengimpor', 'pertumbuhan', 'kecelakaan', 'tertabrak', 'tabrakan', 'menabrak', 'ditabrak', 'tenggelam', 'pesawat', 'simtoma', 'penyakit', 'kesehatan', 'covid', 'korban', 'pasien', 'terjangkit', 'tuberkulosis', 'dbd', 'hiv', 'diabetes', 'hipertensi', 'ginjal', 'jantung', 'melitus', '

100%|██████████| 96/96 [31:06<00:00, 19.45s/it]


2889
('kereta', 'bencana', 'gempa', 'tsunami', 'meletus', 'getaran', 'vulkanik', 'meledak', 'ledakan', 'kebakaran', 'terbakar', 'banjir', 'abrasi', 'amblas', 'kekeringan', 'puting', 'beliung', 'angin', 'anjlok', 'pergerakan', 'tumbang', 'kencang', 'petir', 'lebat', 'hujan', 'longsor', 'genangan', 'meteor', 'meluap', 'dagang', 'berita', 'perdagangan', 'ekonomi', 'miliar', 'neraca', 'impor', 'resesi', 'persen', 'perekonomian', 'keuangan', 'sosio', 'makro', 'bisnis', 'moneter', 'makroekonomi', 'perekonomiannya', 'kesejahteraan', 'pertumbuhan', 'inflasi', 'industrialisasi', 'produktivitas', 'infrastruktur', 'pasar', 'produk', 'pembangunan', 'usaha', 'saham', 'ukm', 'ekspor', 'dieskpor', 'diimpor', 'mengekspor', 'mengimpor', 'pertumbuhan', 'kecelakaan', 'tertabrak', 'tabrakan', 'menabrak', 'ditabrak', 'tenggelam', 'pesawat', 'simtoma', 'penyakit', 'kesehatan', 'covid', 'korban', 'pasien', 'terjangkit', 'tuberkulosis', 'dbd', 'hiv', 'diabetes', 'hipertensi', 'ginjal', 'jantung', 'melitus', '

100%|██████████| 96/96 [1:03:35<00:00, 39.75s/it]


1975
('kereta', 'bencana', 'gempa', 'tsunami', 'meletus', 'getaran', 'vulkanik', 'meledak', 'ledakan', 'kebakaran', 'terbakar', 'banjir', 'abrasi', 'amblas', 'kekeringan', 'puting', 'beliung', 'angin', 'anjlok', 'pergerakan', 'tumbang', 'kencang', 'petir', 'lebat', 'hujan', 'longsor', 'genangan', 'meteor', 'meluap', 'dagang', 'berita', 'perdagangan', 'ekonomi', 'miliar', 'neraca', 'impor', 'resesi', 'persen', 'perekonomian', 'keuangan', 'sosio', 'makro', 'bisnis', 'moneter', 'makroekonomi', 'perekonomiannya', 'kesejahteraan', 'pertumbuhan', 'inflasi', 'industrialisasi', 'produktivitas', 'infrastruktur', 'pasar', 'produk', 'pembangunan', 'usaha', 'saham', 'ukm', 'ekspor', 'dieskpor', 'diimpor', 'mengekspor', 'mengimpor', 'pertumbuhan', 'kecelakaan', 'tertabrak', 'tabrakan', 'menabrak', 'ditabrak', 'tenggelam', 'pesawat', 'simtoma', 'penyakit', 'kesehatan', 'covid', 'korban', 'pasien', 'terjangkit', 'tuberkulosis', 'dbd', 'hiv', 'diabetes', 'hipertensi', 'ginjal', 'jantung', 'melitus', '

...

100%|██████████| 96/96 [30:34<00:00, 19.11s/it]


2231
('kereta', 'bencana', 'gempa', 'tsunami', 'meletus', 'getaran', 'vulkanik', 'meledak', 'ledakan', 'kebakaran', 'terbakar', 'banjir', 'abrasi', 'amblas', 'kekeringan', 'puting', 'beliung', 'angin', 'anjlok', 'pergerakan', 'tumbang', 'kencang', 'petir', 'lebat', 'hujan', 'longsor', 'genangan', 'meteor', 'meluap', 'dagang', 'berita', 'perdagangan', 'ekonomi', 'miliar', 'neraca', 'impor', 'resesi', 'persen', 'perekonomian', 'keuangan', 'sosio', 'makro', 'bisnis', 'moneter', 'makroekonomi', 'perekonomiannya', 'kesejahteraan', 'pertumbuhan', 'inflasi', 'industrialisasi', 'produktivitas', 'infrastruktur', 'pasar', 'produk', 'pembangunan', 'usaha', 'saham', 'ukm', 'ekspor', 'dieskpor', 'diimpor', 'mengekspor', 'mengimpor', 'pertumbuhan', 'kecelakaan', 'tertabrak', 'tabrakan', 'menabrak', 'ditabrak', 'tenggelam', 'pesawat', 'simtoma', 'penyakit', 'kesehatan', 'covid', 'korban', 'pasien', 'terjangkit', 'tuberkulosis', 'dbd', 'hiv', 'diabetes', 'hipertensi', 'ginjal', 'jantung', 'melitus', '

100%|██████████| 96/96 [25:32<00:00, 15.97s/it]


440
('kereta', 'bencana', 'gempa', 'tsunami', 'meletus', 'getaran', 'vulkanik', 'meledak', 'ledakan', 'kebakaran', 'terbakar', 'banjir', 'abrasi', 'amblas', 'kekeringan', 'puting', 'beliung', 'angin', 'anjlok', 'pergerakan', 'tumbang', 'kencang', 'petir', 'lebat', 'hujan', 'longsor', 'genangan', 'meteor', 'meluap', 'dagang', 'berita', 'perdagangan', 'ekonomi', 'miliar', 'neraca', 'impor', 'resesi', 'persen', 'perekonomian', 'keuangan', 'sosio', 'makro', 'bisnis', 'moneter', 'makroekonomi', 'perekonomiannya', 'kesejahteraan', 'pertumbuhan', 'inflasi', 'industrialisasi', 'produktivitas', 'infrastruktur', 'pasar', 'produk', 'pembangunan', 'usaha', 'saham', 'ukm', 'ekspor', 'dieskpor', 'diimpor', 'mengekspor', 'mengimpor', 'pertumbuhan', 'kecelakaan', 'tertabrak', 'tabrakan', 'menabrak', 'ditabrak', 'tenggelam', 'pesawat', 'simtoma', 'penyakit', 'kesehatan', 'covid', 'korban', 'pasien', 'terjangkit', 'tuberkulosis', 'dbd', 'hiv', 'diabetes', 'hipertensi', 'ginjal', 'jantung', 'melitus', 'i

100%|██████████| 96/96 [12:48<00:00,  8.00s/it]


In [7]:
from gensim.models import Phrases
from datetime import datetime

iter_through = [
    'bencana',
    'ekonomi',
    'kecelakaan',
    'kesehatan',
    'kriminalitas',
    'olahraga'
    ]

total_topics = [
    5,
    7,
    6,
    6,
    7,
    5
]

number_words = [
    24,
    18,
    20,
    20,
    18,
    24
]

for topik_id, n in enumerate(iter_through):
    if is_perbandingan == False:
        break
    df_train = joblib.load('D:/Petakabar/perbandingan/' + n + '_train_unpreprocessed.pkl')
    # df_train = joblib.load('D:/Petakabar/perbandingan/' + n + '_test_unpreprocessed.pkl')

    df_train = df_train[['berita_title', 'berita_date', 'berita_desc', 'berita_source']]
    df_train.rename(columns = {'berita_title':'title', 'berita_date':'date', 'berita_desc':'desc', 'berita_source':'source'}, inplace=True)
    
    print(len(df_train))

    for x in range(0, df_train.shape[0]):
        text = preprocessing(df_train.iloc[x, -2], mode = 'lda', topik = n)
        df_train.iloc[x, -2] = (' '.join(text))

    for x in range(0, df_train.shape[0]):
        text = preprocessing(df_train.iloc[x, -4], mode = 'lda', topik = n)
        df_train.iloc[x, -4] = (' '.join(text))

    df_train['desc'] = df_train['desc'].apply(remove_number)
    df_train['desc'] = df_train['desc'].apply(word_tokenize)
    df_train['desc'] = [[token for token in doc if len(token) > 2] for doc in df_train['desc']]
    df_train['title'] = df_train['title'].apply(remove_number)
    df_train['title'] = df_train['title'].apply(word_tokenize)
    df_train['title'] = [[token for token in doc if len(token) > 2] for doc in df_train['title']]
    # df_train['fdist'] = df_train['desc'].apply(FreqDist)

    bigram = Phrases(df_train['desc'], min_count=10)
    for idx in range(len(df_train['desc'])):
        for token in bigram[df_train['desc'].iloc[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                df_train['desc'].iloc[idx].append(token)
                word1, word2 = token.split("_")
                dont_remove = ('kereta', 'bencana', 'gempa', 'tsunami', 'meletus', 'getaran', 'vulkanik', 'meledak', 'ledakan', 'kebakaran',
                    'terbakar', 'banjir', 'abrasi', 'amblas', 'kekeringan', 'puting', 'beliung', 'angin', 'anjlok', 'pergerakan', 'tumbang',
                    'kencang', 'petir', 'lebat', 'hujan', 'longsor', 'genangan', 'meteor', 'meluap', 'dagang', 'berita', 'perdagangan', 'ekonomi',
                    'miliar', 'neraca', 'impor', 'resesi', 'persen', 'perekonomian', 'keuangan', 'sosio', 'makro', 'bisnis', 'moneter', 'makroekonomi',
                    'perekonomiannya', 'kesejahteraan', 'pertumbuhan', 'inflasi', 'industrialisasi', 'produktivitas', 'infrastruktur', 'pasar',
                    'produk', 'pembangunan', 'usaha', 'saham', 'ukm', 'ekspor', 'dieskpor', 'diimpor', 'mengekspor', 'mengimpor', 'pertumbuhan', 
                    'kecelakaan', 'tertabrak', 'tabrakan', 'menabrak', 'ditabrak', 'tenggelam', 'pesawat', 'simtoma', 'penyakit', 'kesehatan', 'covid',
                    'korban', 'pasien', 'terjangkit', 'tuberkulosis', 'dbd', 'hiv', 'diabetes', 'hipertensi', 'ginjal', 'jantung', 'melitus', 'infeksi',
                    'terinfeksi', 'bakteri', 'virus', 'vaksin', 'polio', 'imunisasi', 'meningitis', 'aids', 'diare', 'pneumonia', 'radang', 'pandemi',
                    'corona', 'malaria', 'muntah', 'kejang', 'batuk', 'hipertensi', 'obesitas', 'gizi', 'sars', 'darah', 'pasien', 'influenza', 'flu',
                    'kanker', 'anemia', 'hepatitis', 'liver', 'iritasi', 'pengobatan', 'obat', 'alergi', 'gatal', 'ruam', 'kelainan', 'bunuh', 'pembunuhan',
                    'membunuh', 'dibunuh', 'pencurian', 'curi', 'mencuri', 'dicuri', 'pemerkosaan', 'memperkosa', 'diperkosa', 'perkosa', 'rampok',
                    'perampokan', 'merampok', 'dirampok', 'begal', 'pembegalan', 'membegal', 'dibegal', 'pembacokan', 'bacok', 'membacok', 'dibacok',
                    'penusukan', 'tusuk', 'ditusuk', 'menusuk', 'pukul', 'pemukulan', 'memukul', 'dipukul', 'luka', 'melukai', 'dilukai', 'sepakbola',
                    'bulutangkis', 'basket', 'raket', 'voli', 'futsal', 'tenis', 'turnamen', 'motogp', 'pertandingan', 'pon')
                if word1 not in dont_remove:
                    df_train['desc'].iloc[idx].remove(word1)
                if word2 not in dont_remove:
                    df_train['desc'].iloc[idx].remove(word2)
                # print(token)
    print(dont_remove)
    dictionary = corpora.Dictionary(df_train['desc'])
    dictionary.filter_extremes(no_below=30, no_above=0.85)
    print(dictionary)

    corpus = [dictionary.doc2bow(berita) for berita in df_train['desc']]

    # Creating the object for LDA model using gensim library
    # Lda = gensim.models.ldamodel.LdaModel

    print(topik_id, total_topics[topik_id], number_words[topik_id])
    
    # the_path = PATH + n + "/model/lda_model" + "_" + str(total_topics[topik_id]) + "_" + str(number_words[topik_id])
    the_path = PATH + n + "/model/vis_lda_model" + "_" + str(total_topics[topik_id]) + "_0.01_0.01"
    if not os.path.exists(the_path):
        os.mkdir(the_path)
    temp_file = datapath(the_path + "/lda_model")
    # lda_model = Lda(corpus, num_topics=total_topics[topik_id], id2word=dictionary, passes=20, iterations=100,alpha=[0.01]*total_topics[topik_id], eta=[0.01]*len(dictionary.keys()), random_state=1)
    # lda_model.save(temp_file)

    # lda_model.show_topics(num_topics=total_topics[topik_id], num_words=number_words[topik_id])

    # now = datetime.now() # current date and time
    # date_time = now.strftime("%m-%d-%Y_%H-%M-%S")
    # print("date and time:",date_time)
    lda_model = models.ldamodel.LdaModel.load(temp_file)
    LDAvis_html = '/ldavis_final'
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    with open(PATH + n + LDAvis_html, 'wb') as f:
            pickle.dump(LDAvis_prepared, f)
    pyLDAvis.save_html(LDAvis_prepared, PATH + n + LDAvis_html +'.html')

    #Compute Coherence Score
    # coherence_model_lda = CoherenceModel(model=lda_model, texts=df_train['desc'], dictionary=dictionary, coherence='c_v')
    # coherence_lda = coherence_model_lda.get_coherence()
    # print('Coherence Score:', coherence_lda)
    # print('Perplexity Score:', lda_model.log_perplexity(corpus))
    

    # print(df_train.head(2))

    # supporting function
    def compute_coherence_values(dictionary, k, a, b):
        
        # lda_model = gensim.models.ldamodel.LdaModel( corpus,
        #                                     id2word=dictionary,
        #                                     num_topics=k, 
        #                                     random_state=1,
        #                                     passes=20,
        #                                     iterations=100,
        #                                     alpha=a,
        #                                     eta=b)
        
        the_path = PATH + n + "/model/vis_lda_model" + "_" + str(k) + "_" + str(a) + "_" + str(b)
        if not os.path.exists(the_path):
            os.mkdir(the_path)
        temp_file = datapath(the_path + "/lda_model")
        # lda_model.save(temp_file)
        lda_model = models.ldamodel.LdaModel.load(temp_file)
        
        coherence_model_lda = CoherenceModel(model=lda_model, texts=df_train['desc'], dictionary=dictionary, coherence='u_mass')
        
        return coherence_model_lda.get_coherence(), lda_model.log_perplexity(corpus)

    import tqdm

    # Topics range
    min_topics = 2
    max_topics = 8
    step_size = 1
    topics_range = range(min_topics, max_topics, step_size)

    # Alpha parameter
    alpha = [0.01, 0.1, 1]
    # alpha = list(np.arange(0.01, 1.02, 0.5))
    # alpha.append('symmetric')
    # alpha.append('asymmetric')

    # Beta parameter
    beta = [0.01, 0.1, 1]
    # beta = list(np.arange(0.01, 1.02, 0.5))
    # beta.append('symmetric')

    model_results = {
                    'Topics': [],
                    'Alpha': [],
                    'Beta': [],
                    'Coherence': [],
                    'Perplexity':[]
                    }

    # Can take a long time to run
    if 1 == 0:
        pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)))
        
        # iterate through validation corpuses
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta: 
                    # get the coherence score for the given parameters
                    res = compute_coherence_values(dictionary=dictionary, 
                                                k=k, a=a, b=b)
                    # Save the model results
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(res[0])
                    model_results['Perplexity'].append(res[1])
                    
                    pbar.update(1)
        # pd.DataFrame(model_results).to_csv(prebandingan_path + 'lda_tuning_test_results_'+ n +'.csv', index=False)
        pbar.close()

2565
('kereta', 'bencana', 'gempa', 'tsunami', 'meletus', 'getaran', 'vulkanik', 'meledak', 'ledakan', 'kebakaran', 'terbakar', 'banjir', 'abrasi', 'amblas', 'kekeringan', 'puting', 'beliung', 'angin', 'anjlok', 'pergerakan', 'tumbang', 'kencang', 'petir', 'lebat', 'hujan', 'longsor', 'genangan', 'meteor', 'meluap', 'dagang', 'berita', 'perdagangan', 'ekonomi', 'miliar', 'neraca', 'impor', 'resesi', 'persen', 'perekonomian', 'keuangan', 'sosio', 'makro', 'bisnis', 'moneter', 'makroekonomi', 'perekonomiannya', 'kesejahteraan', 'pertumbuhan', 'inflasi', 'industrialisasi', 'produktivitas', 'infrastruktur', 'pasar', 'produk', 'pembangunan', 'usaha', 'saham', 'ukm', 'ekspor', 'dieskpor', 'diimpor', 'mengekspor', 'mengimpor', 'pertumbuhan', 'kecelakaan', 'tertabrak', 'tabrakan', 'menabrak', 'ditabrak', 'tenggelam', 'pesawat', 'simtoma', 'penyakit', 'kesehatan', 'covid', 'korban', 'pasien', 'terjangkit', 'tuberkulosis', 'dbd', 'hiv', 'diabetes', 'hipertensi', 'ginjal', 'jantung', 'melitus', '